In [1]:
import json
import glob
import os
import pandas as pd
import numpy as np
import pickle
import random

In [2]:
import torch
from torch_geometric.data import Data
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ChebConv, SAGEConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.datasets import TUDataset
import torch.optim.lr_scheduler as lrs
from torch_geometric.loader import DataLoader

/home/patrycja/python_envs/graph_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.manual_seed(12345)
random.seed(12345)

In [4]:
def parse_data(flag, json_path, classes):
    
    counter = 0
    edge_index = []
    lookup = {}
    graphs = []
    
    with open(json_path, "r") as f:
        adjacency_list = json.load(f)

        for l in adjacency_list:
            lookup[l] = counter
            counter+=1

        for l in adjacency_list:
            for e in adjacency_list[l]:
                edge_index.append([lookup[l], lookup[e]])
        for d in os.listdir("data"):
            for f in glob.glob(os.path.join("data", d, flag+"*")):

                try:
                    graph    = {"y": classes[d.split("_")[0]], "edge_index": edge_index, "x":[]}
                    features = pd.read_csv(f, index_col=[0])
                    features = features.replace('', -1, regex=True)

                    for l in lookup:
                        if l.startswith("create_dir_") or l.startswith("cleanup_"):
                            new_l = l.split("-")[0]
                        else:
                            new_l = l                            
                        job_features = features[features.index.str.startswith(new_l)][['type', 'ready',
                                               'submit', 'execute_start', 'execute_end', 'post_script_start',
                                               'post_script_end', 'wms_delay', 'pre_script_delay', 'queue_delay',
                                               'runtime', 'post_script_delay', 'stage_in_delay', 'stage_out_delay']].values.tolist()[0]
                        
                        if len(features[features.index.str.startswith(new_l)])<1:
                            continue
                        if job_features[0]=='auxiliary':
                            job_features[0]= 0
                        if job_features[0]=='compute':
                            job_features[0]= 1
                        if job_features[0]=='transfer':
                            job_features[0]= 2
                        job_features = [-1 if x != x else x for x in job_features]
                        graph['x'].insert(lookup[l], job_features)

                    t_list=[]
                    for i in range(len(graph['x'])):
                        t_list.append(graph['x'][i][1])
                    minim = min(t_list)
                    
                    for i in range(len(graph['x'])):
                        lim = graph['x'][i][1:7]
                        lim =[ v-minim for v in lim]
                        graph['x'][i][1:7] = lim            
                        graphs.append(graph)
                except:
                    print("Error with the file's {} format.".format(f))
    return graphs

In [5]:
def load_data(flag):

    classes = {"normal": 0}
    counter = 1
    json_path = ""
    
    for d in os.listdir("data"):
        d = d.split("_")[0]
        
        if d in classes:
            continue
        classes[d] = counter
        
        counter += 1
        
    if flag == "nowcast-clustering-16":
        json_path = "adjacency_list_dags/casa_nowcast_clustering_16.json"
    elif flag == "1000genome":
        json_path = "adjacency_list_dags/1000genome.json"
    elif flag =="nowcast-clustering-8":
        json_path = "adjacency_list_dags/casa_nowcast_clustering_8.json"
    elif flag == "wind-clustering-casa":
        json_path = "adjacency_list_dags/casa_wind_clustering.json"
    elif flag == "wind-noclustering-casa":
        json_path = "adjacency_list_dags/casa_wind_no_clustering.json"
        
    graphs = parse_data(flag, json_path, classes)
    return graphs

In [6]:
def train(train_loader, model, criterion, optimizer):
    model.train()
    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data.x.float(), data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader, model):
    model.eval()
    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x.float(), data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.

In [128]:
graphs = []
type_names = [ "nowcast-clustering-16","1000genome", "nowcast-clustering-8",
              "wind-clustering-casa","wind-noclustering-casa" ]

for f in type_names:
    flag = "nowcast-clustering-16"
    graphs.append(load_data(flag))

In [7]:
class GCN(torch.nn.Module):
        def __init__(self, hidden_channels):
            super(GCN, self).__init__()
            torch.manual_seed(12345)
            self.conv1 = SAGEConv(14, hidden_channels)
            self.conv2 = SAGEConv(hidden_channels, hidden_channels)
            self.conv3 = SAGEConv(hidden_channels, hidden_channels)
            self.lin = Linear(hidden_channels, 4)

        def forward(self, x, edge_index, batch):
            # 1. Obtain node embeddings 
            x = self.conv1(x, edge_index)
            x = x.relu()
            x = self.conv2(x, edge_index)
            x = x.relu()
            x = self.conv3(x, edge_index)
            # 2. Readout layer
            x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
            # 3. Apply a final classifier
            x = F.dropout(x, p=0.5, training=self.training)
            x = self.lin(x)
            return x

In [9]:
def train_mod(flag, data_read=1):    
    print(flag)
    
    # parse data from raw files
    if data_read ==0:
        graphs = load_data(flag)
        print(len(graphs))
        with open('graph_all_'+ str(flag) + '.pkl','wb') as f:
            pickle.dump(graphs, f)
    else:
        with open('graph_all_'+ str(flag) + '.pkl','rb') as f:
            graphs = pickle.load(f)
    
    y_list = []
    for gr in graphs:
        y_list.append(gr['y'])
    print(min(y_list))
    print(max(y_list))
    print(np.unique(np.array(y_list), return_counts=True))  
    datasets=[]

    for element in graphs:
        gx = torch.tensor(np.array(element['x']) ) 
        ge = torch.tensor(np.array(element['edge_index']) ).T
        gy = torch.tensor(np.array(element['y']).reshape([-1]))

        v_min, v_max = gx.min(), gx.max()
        new_min, new_max = -1, 1
        gx = (gx - v_min)/(v_max - v_min)*(new_max - new_min) + new_min
        datasets.append( Data(x=gx, edge_index=ge, y=gy) )

    dataset = datasets
    random.shuffle(datasets)
    train_dataset = datasets[: int(len(datasets)*0.80) ]
    test_dataset  = datasets[int(len(datasets)*0.80):]
    
    print(f'Number of training graphs: {len(train_dataset)}')
    print(f'Number of test graphs: {len(test_dataset)}')
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)
    
    model = GCN(hidden_channels=64).float()
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = lrs.ExponentialLR(optimizer, gamma=0.9)
    
    for epoch in range(1, 200):
        print(epoch)
        train(train_loader, model, criterion, optimizer)
        train_acc = test(train_loader, model)
        test_acc  = test(test_loader, model)
        
        if epoch%100==0:
            scheduler.step()
            print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epoch%100==0:
            torch.save(model.state_dict(), 'model_'+flag+'_')


In [ ]:
type_names = [ "nowcast-clustering-16","1000genome", "nowcast-clustering-8",
              "wind-clustering-casa","wind-noclustering-casa" ]


In [11]:
train_mod( "nowcast-clustering-16", data_read=0)

nowcast-clustering-16
11070
0
3
(array([0, 1, 2, 3]), array([2430, 2700, 3240, 2700]))
Number of training graphs: 8856
Number of test graphs: 2214
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
Epoch: 100, Train Acc: 0.5894, Test Acc: 0.5772
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
